# IBM Data Science Final Notebook

### First we have to start with scraping Wikipedia for the necessary information

In [1]:
import requests # to handle the url request
from bs4 import BeautifulSoup # to process the request

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
read_in = requests.get(url).text

In [3]:
soup = BeautifulSoup(read_in, 'lxml')

In [4]:
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [5]:
table = soup.find('table')

In [6]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

### Now we can pass the contents to a pandas dataframe

In [7]:
import pandas as pd

df = pd.DataFrame()
df = df.append(table_contents, ignore_index = True)

In [8]:
df.shape

(103, 3)

## We can read in the contents of the provided csv to get cooridnates for each postal code

In [9]:
Coords = pd.read_csv(r'C:\Users\emcollin\OneDrive - Volusia County Schools\Documents\Projects\IBM Data Science Final\Geospatial_Coordinates.csv')
Coords.rename(columns={'Postal Code':'PostalCode'},inplace=True)
Coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## And we do a join on 'Postal Code'

In [10]:
df = pd.merge(df, Coords, on = 'PostalCode')

In [11]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## From here, we will cluster the 'York' Boroughs

In [12]:
yorks = df[df['Borough'].str.contains('York',regex=False)]
yorks.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [13]:
yorks.shape

(34, 5)

In [14]:
import folium

In [15]:
yorks_map = folium.Map(location=[43.753259,-79.329656],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(yorks['Latitude'],yorks['Longitude'],yorks['Borough'],yorks['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(yorks_map)
yorks_map

## And we can use cluster analysis for the neighborhoods

In [16]:
from sklearn.cluster import KMeans

k=6
yorks_cluster = yorks.drop(['PostalCode','Borough','Neighborhood'],1)
km = KMeans(n_clusters = k,random_state=0).fit(yorks_cluster)
km.labels_
yorks.insert(0, 'Cluster Labels', km.labels_)

In [17]:
yorks.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,0,M4A,North York,Victoria Village,43.725882,-79.315572
3,1,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,0,M3B,North York,Don Mills North,43.745906,-79.352188
8,0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


### And we plot based on the cluster label

In [18]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

york_clusters = folium.Map(location=[43.753259,-79.329656],zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighbourhood, cluster in zip(yorks['Latitude'], yorks['Longitude'], yorks['Neighborhood'], yorks['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(york_clusters)
       
york_clusters